In [ ]:
# 네이버 검색 API예제는 블로그를 비롯 전문자료까지 호출방법이 동일하므로 blog검색만 대표로 예제를 올렸습니다.
# 네이버 검색 Open API 예제 - 블로그 검색
import os
import sys
import urllib.request
client_id = "5jzCiWVGf6GeodQ9KwUV"
client_secret = "ySMUelE3jF"
encText = urllib.parse.quote("마스크")
url = "https://openapi.naver.com/v1/search/shop?query=" + encText +"&display=20" # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200): # 요청 성공
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
import urllib.request
import datetime
import time
import json

client_id = "5jzCiWVGf6GeodQ9KwUV"
client_secret = "ySMUelE3jF"

In [ ]:
def getRequestUrl(url):
    req=urllib.request.Request(url)
    req.add_header('X-Naver-Client-Id',client_id)
    req.add_header('X-Naver-Client-Secret',client_secret)
    
    try:
        response=urllib.request.urlopen(req)
        if response.getcode()==200:
            print("[%s] Url Request Success"%datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
    print("[%s] Error for Url : %s"%(datetime.datetime.now(),url))
    return None # 1만 건 이상 요청할 시 error 발생

In [ ]:
def getNaverSearch(node, srcText, start, display):
    base="https://openapi.naver.com/v1/search"
    node="/%s.json"%node
    parameters="?query=%s&start=%s&display=%s"%(urllib.parse.quote(srcText),start,display)
    url=base+node+parameters
    responseDecode=getRequestUrl(url)
    
    if responseDecode==None:
        return None
    else:
        return json.loads(responseDecode)

In [ ]:
def getPostData(item, jsonResult, cnt):
    title=item['title'].strip()
    description=item['description'].strip()
    org_link=item['originallink']
    link=item['link']
    
    pDate=datetime.datetime.strptime(item['pubDate'],'%a, %d %b %Y %H:%M:%S +0900')
    pDate=pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    #뉴스 검색 결과를 jsonResult에
    jsonResult.append({'cnt':cnt,'title':title,'description':description,
                       'org_link':org_link,'link':link,'pDate':pDate})
    
    #dictionary가 아닌 tuple로 저장하면 DataFrame을 생성할 수 있다
    return

In [ ]:
def main():
    node='news'
    srcText=input('검색어를 입력해주세요')
    cnt=0;
    jsonResult=[]
    
    jsonResponse=getNaverSearch(node, srcText, 1, 10)
    total=jsonResponse['total']
    #print(total)
    #print(jsonResponse)
    
    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        # 데이터 저장
        for item in jsonResponse['items']:
            cnt+=1
            getPostData(item, jsonResult, cnt)

        start=jsonResponse['start']+jsonResponse['display']
        jsonResponse=getNaverSearch(node, srcText, start, 10)
        
        #print(jsonResult)
        
    with open('%s_naver_%s.json'%(srcText,node),'w',encoding='utf-8') as outfile:
        jsonFile=json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)
        
    print("전체 데이터 건수 %d건에서 가져온 데이터 %d건입니다"%(total,cnt))
    print('%s_naver_%s.json SAVE'%(srcText,node))

In [ ]:
main()

In [ ]:
#상품정보 검색 결과로 변경해보기
import os
import sys
import urllib.request
import pandas as pd
client_id = "5jzCiWVGf6GeodQ9KwUV"
client_secret = "ySMUelE3jF"

def getRequestUrl(url):
    req=urllib.request.Request(url)
    req.add_header('X-Naver-Client-Id',client_id)
    req.add_header('X-Naver-Client-Secret',client_secret)
    
    try:
        response=urllib.request.urlopen(req)
        if response.getcode()==200:
            print("[%s] Url Request Success"%datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
    print("[%s] Error for Url : %s"%(datetime.datetime.now(),url))
    return None # 1만 건 이상 요청할 시 error 발생

In [ ]:
def getNaverSearch(node, srcText, start, display):
    base="https://openapi.naver.com/v1/search"
    node="/%s.json"%node
    parameters="?query=%s&start=%s&display=%s"%(urllib.parse.quote(srcText),start,display)
    url=base+node+parameters
    responseDecode=getRequestUrl(url)
    
    if responseDecode==None:
        return None
    else:
        return json.loads(responseDecode)

In [ ]:
def getPostData(item, jsonResult, cnt):
    productId=item['productId']
    title=item['title'].strip()
    brand=item['brand']
    lprice=item['lprice']
    link=item['link']
    
    #pDate=datetime.datetime.strptime(item['pubDate'],'%a, %d %b %Y %H:%M:%S +0900')
    #pDate=pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    #상품 검색 결과를 jsonResult에
    #shop_list=(productId,title,brand,lprice,link)
    jsonResult.append((productId,title,brand,lprice,link))
    
    #dictionary가 아닌 tuple로 저장하면 DataFrame을 생성할 수 있다
    return

In [ ]:
def main():
    node='shop'
    srcText=input('검색어를 입력해주세요')
    cnt=0;
    jsonResult=[]
    
    jsonResponse=getNaverSearch(node, srcText, 1, 10)
    total=jsonResponse['total']
    #print(total)
    #print(jsonResponse)
    
    
    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        # 데이터 저장
        shop_list=[]
        for item in jsonResponse['items']:
            cnt+=1
            getPostData(item, jsonResult, cnt)

        start=jsonResponse['start']+jsonResponse['display']
        jsonResponse=getNaverSearch(node, srcText, start, 10)
        
        #print(jsonResult)
        
    with open('%s_naver_%s.json'%(srcText,node),'w',encoding='utf-8') as outfile:
        jsonFile=json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)
        
    print("전체 데이터 건수 %d건에서 가져온 데이터 %d건입니다"%(total,cnt))
    print('%s_naver_%s.json SAVE'%(srcText,node))
    
    df=pd.DataFrame(jsonResult,columns=['productId','title','brand','lprice','link'])
    df.to_csv('data/naver_shop.csv', index=False)
    df

In [ ]:
main()

# papago
- 번역기

In [6]:
import os
import sys
import urllib.request
import json
client_id = "NOk_F9opvpvXdWmQNjmt" # 개발자센터에서 발급받은 Client ID 값
client_secret = "E0JYpfzj17" # 개발자센터에서 발급받은 Client Secret 값

def papago_func(encText):
    encText = urllib.parse.quote(encText)
    data = "source=ko&target=ja&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read().decode('utf-8')

        #json으로 저장
        jsonData=json.loads(response_body)

        #print(response_body.decode('utf-8'))
        print(jsonData)
        print(jsonData['message']['result']['translatedText'])
    else:
        print("Error Code:" + rescode)

encText=input("번역할 문장을 입력하세요")
papago_func(encText)


번역할 문장을 입력하세요 안녕


{'message': {'result': {'srcLangType': 'ko', 'tarLangType': 'ja', 'translatedText': 'こんにちは', 'engineType': 'PRETRANS', 'pivot': None, 'dict': None, 'tarDict': None}, '@type': 'response', '@service': 'naverservice.nmt.proxy', '@version': '1.0.0'}}
こんにちは
